In [3]:
import pickle
import json
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import signal
from urllib.parse import urlparse
import uuid

In [4]:
all_needed_comments = pickle.load(open('../data_2017-09/all_needed_comments.pkl', 'rb'))
url_file_map = pickle.load(open('../data_2017-09/url_file_map.pkl', 'rb'))

In [6]:
for url in url_file_map:
    if 'status' not in url_file_map[url] or url_file_map[url]['status'] == "failure": continue
    print(url_file_map[url])

{'filename': 'fb78e1439cb34d528bc3b7c63aa6d690', 'chain': ['dmehrxc', 'dmehs40'], 'status': 'success', 'split': 'train'}
{'filename': '3d549be0eabc4b498124154e6042f5f8', 'chain': ['dn3oifl', 'dn43n9m', 'dn4oi9w', 'dn4pk6l', 'dn4q8gi'], 'status': 'success', 'split': 'train'}
{'filename': '3b7659e157dd45e0b601b310810dcad7', 'chain': ['dmehvny', 'dmf2f2k'], 'status': 'success', 'split': 'train'}
{'filename': 'd6065fd09a824e0c9c0bbf88d1292d64', 'chain': ['dmlcnr6', 'dmldwiv', 'dmljki6', 'dmlktlo', 'dmlux93', 'dmm08c0', 'dmmdyi0', 'dmmrj76'], 'status': 'success', 'split': 'train'}
{'filename': '7122309c6dd34afe841ee96c81e90df1', 'chain': ['dnff170', 'dnfk9x8', 'dnfqxg7'], 'status': 'success', 'split': 'train'}
{'filename': 'e6f5e33b73854cbeab322ea662e3f837', 'chain': ['dmyukpm', 'dmz69p7', 'dmz7jfk', 'dmzdd51', 'dmzdd91'], 'status': 'success', 'split': 'train'}
{'filename': '497be1d2ea884a0eb50fd5227a5fd96a', 'chain': ['dmeht2s', 'dmei74p'], 'status': 'success', 'split': 'test'}
{'filename'